In [1]:
from typing import List, Optional, Dict, Any
from pydantic import BaseModel, Field
from langchain_core.messages import BaseMessage, AIMessage, HumanMessage
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_core.runnables import Runnable, RunnableLambda
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_openai import ChatOpenAI

In [2]:
class InMemoryHistory(BaseChatMessageHistory, BaseModel):
    messages: List[BaseMessage] = Field(default_factory=list)
    save_mode: Optional[str] = Field(default="both")  # "input", "output", "both"
    
    def add_messages(self, messages: List[BaseMessage]) -> None:
        """조건에 따라 메시지를 저장"""
        if self.save_mode == "input":
            input_messages = [msg for msg in messages if isinstance(msg, HumanMessage)]
            self.messages.extend(input_messages)
        elif self.save_mode == "output":
            output_messages = [msg for msg in messages if isinstance(msg, AIMessage)]
            self.messages.extend(output_messages)
        elif self.save_mode == "both":
            self.messages.extend(messages)

    def clear(self) -> None:
        self.messages = []

In [3]:
store = {}
def get_session_history(session_id: str, save_mode: str = "both") -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = InMemoryHistory(save_mode=save_mode)
    return store[session_id]

In [4]:
class RunnableWithMessageHistory(Runnable):
    def __init__(self, runnable: Runnable, get_session_history, input_messages_key: str, history_messages_key: str, context_key: Optional[str] = None):
        self.runnable = runnable
        self.get_session_history = get_session_history
        self.input_messages_key = input_messages_key
        self.history_messages_key = history_messages_key
        self.context_key = context_key
    
    def invoke(self, input: Dict[str, Any], config: Optional[Dict[str, Any]] = None) -> Any:
        session_id = config["configurable"]["session_id"]
        save_mode = config["configurable"].get("save_mode", "both")
        history = self.get_session_history(session_id, save_mode)
        
        current_input = input[self.input_messages_key]
        
        if isinstance(current_input, str):
            current_input_message = HumanMessage(content=current_input)
        
        input[self.history_messages_key] = history.messages
        
        result = self.runnable.invoke(input, config)
        
        if self.context_key and input.get(self.context_key):
            context = input[self.context_key]
            result_with_context = AIMessage(content=f"{context}\n{result.content}")
            history.add_messages([current_input_message, result_with_context])
        else:
            history.add_messages([current_input_message, result])
        
        return result

In [6]:
def add_memory(runnable, session_id, context="", save_mode="both"):
    runnable_with_memory = RunnableWithMessageHistory(
        runnable,
        get_session_history,
        input_messages_key="input",
        history_messages_key="chat_history",
        context_key="context"
    )
    
    memory_by_session = RunnableLambda(
        lambda input: runnable_with_memory.invoke(
            {**input, "context": context},
            config={"configurable": {"session_id": session_id,
                                     "save_mode": save_mode}}
        )
    )
    return memory_by_session

In [7]:
message_type_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            """
            You are a robot that classifies customer input messages into one of the following two types:
            - Product inquiry, order history inquiry, order change history inquiry, order cancellation history inquiry: '문의'
            - Order request, order change request, order cancellation request: '요청'
            
            You need to review the messages in the Messages Placeholder from the latest to the oldest.

            Consider the previous AI responses and their classifications to understand the intent behind the current input. 
            Use this context to make an accurate classification. 
            If the latest AI response was classified as '요청', and the current input is related to an order, it is likely a '요청'.
            
            Additionally, if the input contains order details, it should be classified as '요청'.
            """
        ),
        MessagesPlaceholder(variable_name="chat_history"),
        ("human", "{input}"),
    ]
)

model = ChatOpenAI()
classify_message_chain = message_type_prompt | model

In [25]:
runnable_with_memory = add_memory(classify_message_chain, session_id="test1", save_mode="both")
print(runnable_with_memory.invoke({"input": "주문 취소할게"}, config={"configurable": {"session_id": "test1"}}))
print(store["test1"].messages)

content='요청' response_metadata={'token_usage': {'completion_tokens': 3, 'prompt_tokens': 202, 'total_tokens': 205}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None} id='run-4af717de-1e1c-4b1a-afa1-25e126824a67-0'
[HumanMessage(content='주문 취소할게'), AIMessage(content='\n요청'), HumanMessage(content='주문 취소할게'), AIMessage(content='요청', response_metadata={'token_usage': {'completion_tokens': 3, 'prompt_tokens': 202, 'total_tokens': 205}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-4af717de-1e1c-4b1a-afa1-25e126824a67-0')]


In [26]:
runnable_with_memory = add_memory(classify_message_chain, session_id="test1", context="사용자 입력 유형", save_mode="both")
print(runnable_with_memory.invoke({"input": "주문 변경도 가능해?"}, config={"configurable": {"session_id": "test1"}}))
print(store["test1"].messages)

content='요청' response_metadata={'token_usage': {'completion_tokens': 3, 'prompt_tokens': 220, 'total_tokens': 223}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None} id='run-5afcd8d0-0eaf-4946-bc6c-17aeda7b378b-0'
[HumanMessage(content='주문 취소할게'), AIMessage(content='\n요청'), HumanMessage(content='주문 취소할게'), AIMessage(content='요청', response_metadata={'token_usage': {'completion_tokens': 3, 'prompt_tokens': 202, 'total_tokens': 205}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-4af717de-1e1c-4b1a-afa1-25e126824a67-0'), HumanMessage(content='주문 변경도 가능해?'), AIMessage(content='사용자 입력 유형\n요청')]


In [8]:
runnable_with_memory = add_memory(classify_message_chain, session_id="test1", context="사용자 입력 유형", save_mode="output")
print(runnable_with_memory.invoke({"input": "주문 변경할게요"}, config={"configurable": {"session_id": "output_only"}}))
print(store["output_only"].messages)

content='요청' response_metadata={'token_usage': {'completion_tokens': 3, 'prompt_tokens': 182, 'total_tokens': 185}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None} id='run-c9251373-fcf5-4198-9664-cc914dbd3db3-0'


KeyError: 'output_only'

In [9]:
runnable_with_memory = add_memory(classify_message_chain, session_id="output_only", context="사용자 입력 유형", save_mode="output")
print(runnable_with_memory.invoke({"input": "주문 변경할게요"}, config={"configurable": {"session_id": "output_only"}}))
print(store["output_only"].messages)

content='요청' response_metadata={'token_usage': {'completion_tokens': 3, 'prompt_tokens': 182, 'total_tokens': 185}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None} id='run-913753c5-4a53-4f85-b773-c9348b4ce5e6-0'
[AIMessage(content='사용자 입력 유형\n요청')]


In [10]:
runnable_with_memory = add_memory(classify_message_chain, session_id="output_only", context="사용자 입력 유형", save_mode="output")
print(runnable_with_memory.invoke({"input": "주문 취소"}, config={"configurable": {"session_id": "output_only"}}))
print(store["output_only"].messages)

content='요청' response_metadata={'token_usage': {'completion_tokens': 3, 'prompt_tokens': 197, 'total_tokens': 200}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None} id='run-23573e98-adf9-48dd-8fdb-3ad8a46aa0be-0'
[AIMessage(content='사용자 입력 유형\n요청'), AIMessage(content='사용자 입력 유형\n요청')]


configurable로 session_id 변경

In [14]:
runnable_with_memory = add_memory(classify_message_chain, session_id="output_only", context="사용자 입력 유형", save_mode="output")
print(runnable_with_memory.invoke({"input": "판매 상품 알려줘"}, config={"configurable": {"session_id": "test1"}}))
print(store["test1"].messages)

content='문의' response_metadata={'token_usage': {'completion_tokens': 2, 'prompt_tokens': 211, 'total_tokens': 213}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None} id='run-1ba7337e-16c4-46ac-acd0-bb0d42e5c692-0'
[HumanMessage(content='주문 취소할게'), AIMessage(content='요청', response_metadata={'token_usage': {'completion_tokens': 3, 'prompt_tokens': 183, 'total_tokens': 186}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-64719567-142e-4ee1-adee-6963babb05ca-0'), HumanMessage(content='주문 변경도 가능해?'), AIMessage(content='요청', response_metadata={'token_usage': {'completion_tokens': 3, 'prompt_tokens': 201, 'total_tokens': 204}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-5b6e5bbe-6a7a-4084-a8ae-02ebd756366f-0'), HumanMessage(content='주문 변경할게요'), AIMessage(content='요청', response_metadata={'token_usage': {'completio